# Mnist_nn_Deep_softmax
- 5개의 layers

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [5]:
# nn layers
linear1 = nn.Linear(784,512,bias = True)
linear2 = nn.Linear(512,512,bias = True)
linear3 = nn.Linear(512,512,bias = True)
linear4 = nn.Linear(512,512,bias = True)
linear5 = nn.Linear(512,10,bias = True)
relu = nn.ReLU()

In [15]:
# weight init
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)
nn.init.xavier_uniform_(linear4.weight)
nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0612,  0.0887, -0.1066,  ...,  0.0463, -0.0457,  0.0222],
        [-0.0815,  0.0365,  0.0433,  ...,  0.0514,  0.0159,  0.0125],
        [-0.0795, -0.0663, -0.1013,  ...,  0.0972, -0.1031,  0.0972],
        ...,
        [ 0.0339, -0.0616, -0.0806,  ...,  0.0397, -0.0559,  0.0483],
        [-0.0347,  0.0078, -0.0569,  ...,  0.0641, -0.0780, -0.0289],
        [-0.0063,  0.0665,  0.0067,  ..., -0.1057,  0.0611,  0.0680]],
       requires_grad=True)

In [17]:
#model
model = torch.nn.Sequential(linear1, relu,
                            linear2, relu, 
                            linear3, relu,
                            linear4, relu,
                            linear5).to(device)
# loss/cost optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [18]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.213038415
Epoch: 0002 cost = 0.090830982
Epoch: 0003 cost = 0.062177073
Epoch: 0004 cost = 0.048546061
Epoch: 0005 cost = 0.041131224
Epoch: 0006 cost = 0.033022325
Epoch: 0007 cost = 0.030975746
Epoch: 0008 cost = 0.025795324
Epoch: 0009 cost = 0.025894376
Epoch: 0010 cost = 0.019369053
Epoch: 0011 cost = 0.018355379
Epoch: 0012 cost = 0.017265452
Epoch: 0013 cost = 0.017509585
Epoch: 0014 cost = 0.018314889
Epoch: 0015 cost = 0.012970344
Learning finished


In [19]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9800000190734863
Label:  8
Prediction:  8
